In [64]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width, empty):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.empty     = empty
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
        
        if empty == True :
            self.neighbors = {(x,y): {(x,y+1),(x+1,y)} for x in range(height) for y in range (width)}
            for x in range(height):
                self.neighbors[x,width-1].remove((x,width))
        
        
        
    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    
    def add_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire

    
    def remove_wall(self,c1,c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # retirer le mur
        if c2 not in self.neighbors[c1]:
            self.neighbors[c1].add(c2)
        if c1 not in self.neighbors[c2]:
            self.neighbors[c2].add(c1)
    
    
    def get_walls(self):
        """
        Retourne la liste de tous les murs sous la forme d’une liste de tuple de cellules
        """
        murs = []
        for i in range(laby.height):
            for j in range(laby.width):
                c1=(i,j)
                for c2 in self.neighbors[c1]:
                    if c1 < c2:  # On ne compte chaque mur qu'une seule fois (par exemple, on ne compte pas à la fois le mur (c1,c2) et le mur (c2,c1))
                        murs.append((c1, c2))
        return murs



    def fill(self):      
        """
        Ajoute tous les murs possibles dans le labyrinthe
        """
        for i in range(laby.height):
            for j in range(laby.width):
                c1=(i,j)
                for c2 in self.get_contiguous_cells(c1):
                    self.add_wall(c1, c2)
    
    
    def get_contiguous_cells(self, c):
        """
        Retourne la liste des cellules contigües à c dans la grille (sans s’occuper des éventuels murs)
        """
        i, j = c
        voisins = []
        if i > 0:
            voisins.append((i-1, j))
        if i < self.height-1:
            voisins.append((i+1, j))
        if j > 0:
            voisins.append((i, j-1))
        if j < self.width-1:
            voisins.append((i, j+1))
        return voisins
    
    
    def get_reachable_cells(self, c):  # ne marche pas
        """
        Retourne la liste des cellules accessibles depuis c (c’est-à-dire les cellules contiguës à c qui sont dans le voisinage de c)
        """
        reachable = []
        for voisin in self.get_contiguous_cells(c):
            if voisin in self.neighbors[c]:
                reachable.append(voisin)
        return reachable
    
    
    def emptys(self):
        """
        Supprime tous les murs du labyrinthe
        """
        for i in range(laby.height):
            for j in range(laby.width):
                c1=(i,j)
                for c2 in self.get_contiguous_cells(c1):
                    self.remove_wall(c1, c2)
    
    def get_cells(self):
        """
        Retourne la liste de toutes les cellules de la grille du labyrinthe.
        """
        L = []
        for i in range(laby.height):
            for j in range(laby.width):
                L.append((i,j))
        return L
    
    def gen_btree(height,width): # ne marche pas 
        laby = Maze(height, width, False) # un labyrinthe plein (contenant tous les murs possibles)
        for i in range(laby.height):
            for j in range(laby.width):
                c1 = (i,j)
                voisins_EST = (i,j+1) # voisins EST
                voisins_SUD = (i+1,j) # voisins SUD
                print(voisins_EST)
                
                if voisins_EST in laby.get_reachable_cells(c1) and voisins_SUD in laby.get_reachable_cells(c1): # si le mur EST et SUD existe
                    aleatoire = randint(1,2)
                    if aleatoire == 1: # supprimer le mur EST
                        laby.remove_wall(c1,voisins_EST)
                    else: # supprimer le mur SUD
                        laby.remove_wall(c1,voisins_SUD)
                    
                elif voisins_EST in laby.get_reachable_cells(c1): # si le mur EST existe mais pas le mur SUD, supprimer le mur EST
                    laby.remove_wall(c1,voisins_EST)
                    
                elif voisins_SUD in laby.get_reachable_cells(c1): # si le mur SUD existe mais pas le mur EST, supprimer le mur SUD
                    laby.remove_wall(c1,voisins_SUD)
        
        return laby
                        
    
    

In [65]:
laby = Maze.gen_btree(4, 4)
print(laby)

(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [3]:
laby = Maze(4, 4, False)
print(laby.info())

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set()}
- Structure cohérente



In [4]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [5]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [6]:
laby.neighbors[(1,3)].remove((2,3))
laby.neighbors[(2,3)].remove((1,3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [7]:
laby.neighbors[(1, 3)].add((2, 3))
laby.neighbors[(2, 3)].add((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [8]:
laby.neighbors[(1, 3)].remove((2, 3))
print(laby)
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}
- Structure incohérente : (2, 3) X (1, 3)



In [9]:
laby = Maze(4, 4, empty = True)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



In [10]:
laby = Maze(4, 4, empty = False)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [11]:
laby = Maze(5, 5, empty = True)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [12]:
laby.add_wall((0,0), (0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [39]:
laby = Maze(5, 5, empty = True)
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [40]:
laby.remove_wall((0, 0), (0, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [41]:
laby.emptys()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

(1, 0)
(0, 1)
(1, 1)
(0, 0)
(0, 2)
(1, 2)
(0, 1)
(0, 3)
(1, 3)
(0, 2)
(0, 4)
(1, 4)
(0, 3)
(0, 0)
(2, 0)
(1, 1)
(0, 1)
(2, 1)
(1, 0)
(1, 2)
(0, 2)
(2, 2)
(1, 1)
(1, 3)
(0, 3)
(2, 3)
(1, 2)
(1, 4)
(0, 4)
(2, 4)
(1, 3)
(1, 0)
(3, 0)
(2, 1)
(1, 1)
(3, 1)
(2, 0)
(2, 2)
(1, 2)
(3, 2)
(2, 1)
(2, 3)
(1, 3)
(3, 3)
(2, 2)
(2, 4)
(1, 4)
(3, 4)
(2, 3)
(2, 0)
(4, 0)
(3, 1)
(2, 1)
(4, 1)
(3, 0)
(3, 2)
(2, 2)
(4, 2)
(3, 1)
(3, 3)
(2, 3)
(4, 3)
(3, 2)
(3, 4)
(2, 4)
(4, 4)
(3, 3)
(3, 0)
(4, 1)
(3, 1)
(4, 0)
(4, 2)
(3, 2)
(4, 1)
(4, 3)
(3, 3)
(4, 2)
(4, 4)
(3, 4)
(4, 3)
┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [18]:
print(laby.get_walls())

[((0, 0), (0, 1)), ((4, 0), (5, 0)), ((4, 1), (5, 1)), ((4, 2), (5, 2)), ((4, 3), (5, 3)), ((4, 4), (5, 4))]


In [19]:
print(laby.get_contiguous_cells((0,1)))

[(1, 1), (0, 0), (0, 2)]


In [66]:
print(laby.get_reachable_cells((0,1)))

[]


In [92]:
laby = Maze.gen_btree(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

